In [2]:
import pysam
import os
import sys
from sys import getsizeof
import time
import pandas as pd
import polars as pl
from collections import defaultdict

from collections import defaultdict
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm


from glob import glob

directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from read_process import incorporate_replaced_pos_info,incorporate_insertions_and_deletions,\
get_positions_from_md_tag,reverse_complement,get_edit_information,get_edit_information_wrapper,\
has_edits,get_total_coverage_for_contig_at_position,\
print_read_info, get_read_information, get_hamming_distance, remove_softclipped_bases,find

from utils import get_contigs_that_need_bams_written, get_contig_lengths_dict, get_intervals, index_bam, write_rows_to_info_file, write_header_to_edit_info, \
write_read_to_bam_file, remove_file_if_exists, make_folder, concat_and_write_bams_wrapper, make_edit_finding_jobs

from core import run_edit_identifier, run_bam_reconfiguration

import os, psutil


In [3]:
multiprocessing.cpu_count()

64

# Preload which barcodes to use...

In [4]:
barcodes_list_path = '/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/cellranger/results/ms_hippo_stamp_EIF4A_batch2/outs/filtered_feature_bc_matrix/barcodes.tsv.gz'

In [5]:
barcode_whitelist = set(pd.read_csv(barcodes_list_path, names=['barcodes']).barcodes.tolist())

In [6]:
len(barcode_whitelist)

39200

# ~~~~~~~~~~~~~~~~~~
# Multi-processing enabled
# ~~~~~~~~~~~~~~~~~~

# An example on a full 10x bam

#### in 10X's bam file, xf=25 means that read is uniquely mapped to a genome, and was used for counting UMI. So we should only look at reads with xf=25 from the 10X bam.

In [7]:
#bampath = '/projects/ps-yeolab5/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/filtered_possorted_ms_hippo_stamp_bam/filtered_keep_xf25_possorted_genome_with_header.bam_MD.bam'
bampath = '/projects/ps-yeolab3/ekofman/sailor2/data/groups_0_1_2_3_4_5_6_7_8_9_10_11_merged.bam'

samfile = pysam.AlignmentFile(bampath, "rb")
samfile_header = str(samfile.header)

output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'

# Print info?
verbose = False 


# How many subcontigs to split each contig into to leverage multi-processing
num_intervals_per_contig = 16

# Go through every read and identify all edits

In [8]:


overall_label_to_list_of_contents, results, overall_time, overall_total_reads, total_seconds_for_reads = run_edit_identifier(
    bampath, 
    output_folder, 
    barcode_whitelist,
    #contigs=['1', '2'],
    num_intervals_per_contig=num_intervals_per_contig, 
    verbose=verbose)


	Contig 1
	Contig 10
	Contig 11
	Contig 12
	Contig 13
	Contig 14
	Contig 15
	Contig 16
	Contig 17
	Contig 18
	Contig 19
	Contig 2
	Contig 3
	Contig 4
	Contig 5
	Contig 6
	Contig 7
	Contig 8
	Contig 9
	Contig MT
	Contig X
	Contig Y
352 total jobs


  8%|██████▌                                                                           | 28/352 [01:06<12:49,  2.38s/it]


KeyboardInterrupt: 

In [9]:
print("Total time: {} seconds".format(overall_time))
print("Total time: {} minutes".format(overall_time/60))

Total time: 675.2126092836261 seconds
Total time: 11.253543488060435 minutes


Memory: 230 Gigabytes

# More helper functions

# Combine all of the reads (string representation) for each barcode
## Groups the results from each sub-contig segment above, for example the reads from the first half of chr1 and those from the second half.

### which finished?

In [ ]:
# Make a subfolder into which the split bams will be placed
split_bams_folder = '{}/split_bams'.format(output_folder)
make_folder(split_bams_folder)
    

In [ ]:
from glob import glob

contigs_to_generate_bams_for = get_contigs_that_need_bams_written(overall_label_to_list_of_contents, split_bams_folder)

In [ ]:
contigs_to_generate_bams_for

### Generate list of jobs to be multiprocessed

In [ ]:
from collections import OrderedDict
import time



In [ ]:
total_bam_generation_time = run_bam_reconfiguration(split_bams_folder, bampath, overall_label_to_list_of_contents, contigs_to_generate_bams_for)

In [ ]:
print("Total time to concat and write bams: {} minutes".format(round(total_bam_generation_time/60)))

# Second loop to get coverage at sites with edits

In [10]:
from glob import glob
import os
import sys
from collections import defaultdict
import pandas as pd
import polars as pl
from multiprocessing import get_context
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm
import numpy as np



directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from utils import get_edit_info_for_barcode_in_contig_wrapper

from core import gather_edit_information_across_subcontigs, run_coverage_calculator

#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'


### Gather the edit information generated for each subcontig, and group by contig so we only have 1 edit information dataframe to process per contig

In [11]:
edit_info_grouped_per_contig_combined = gather_edit_information_across_subcontigs(output_folder)

Grouping edit information outputs by contig...
	0/352...
	10/352...
	20/352...
	30/352...
	40/352...
	50/352...
	60/352...
	70/352...
	80/352...
	90/352...
	100/352...
	110/352...
	120/352...
	130/352...
	140/352...
	150/352...
	160/352...
	170/352...
	180/352...
	190/352...
	200/352...
	210/352...
	220/352...
	230/352...
	240/352...
	250/352...
	260/352...
	270/352...
	280/352...
	290/352...
	300/352...
	310/352...
	320/352...
	330/352...
	340/352...
	350/352...
Done grouping! Concatenating ...
Done concatenating!


### Get coverage at edit positions for each contig

##### Merge across contigs for each barcode???

### This is going at rate of 1 items per seconds... which would take several hours...

In [12]:

results, total_time = run_coverage_calculator(edit_info_grouped_per_contig_combined, output_folder)

 76%|███████████████████████████████████████████████████████████████▉                    | 67/88 [5:43:11<2:48:00, 480.02s/it]

Contig 12_G. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig/split_bams/12_G/12_G.bam.sorted.bam bamfile...
Contig 12_G. Loaded bamfile...
Contig 12_G. Iterating through barcodes...
0/9346 barcodes for 12_G...
300/9346 barcodes for 12_G...
600/9346 barcodes for 12_G...
900/9346 barcodes for 12_G...
1200/9346 barcodes for 12_G...
1500/9346 barcodes for 12_G...
1800/9346 barcodes for 12_G...
2100/9346 barcodes for 12_G...
2400/9346 barcodes for 12_G...
2700/9346 barcodes for 12_G...
3000/9346 barcodes for 12_G...
3300/9346 barcodes for 12_G...
3600/9346 barcodes for 12_G...
3900/9346 barcodes for 12_G...
4200/9346 barcodes for 12_G...
4500/9346 barcodes for 12_G...
4800/9346 barcodes for 12_G...
5100/9346 barcodes for 12_G...
5400/9346 barcodes for 12_G...
5700/9346 barcodes for 12_G...
6000/9346 barcodes for 12_G...
6300/9346 barcodes for 12_G...
6600/9346 barcodes for 12_G...
6900/9346 barcodes for 12_G...
7200/9346 barcodes for 12_G...
7500/9346 barcode

100%|██████████████████████████████████████████████████████████████████████████████████████| 88/88 [6:15:31<00:00, 256.04s/it]


Contig 11_C. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig/split_bams/11_C/11_C.bam.sorted.bam bamfile...
Contig 11_C. Loaded bamfile...
Contig 11_C. Iterating through barcodes...
0/9697 barcodes for 11_C...
300/9697 barcodes for 11_C...
600/9697 barcodes for 11_C...
900/9697 barcodes for 11_C...
1200/9697 barcodes for 11_C...
1500/9697 barcodes for 11_C...
1800/9697 barcodes for 11_C...
2100/9697 barcodes for 11_C...
2400/9697 barcodes for 11_C...
2700/9697 barcodes for 11_C...
3000/9697 barcodes for 11_C...
3300/9697 barcodes for 11_C...
3600/9697 barcodes for 11_C...
3900/9697 barcodes for 11_C...
4200/9697 barcodes for 11_C...
4500/9697 barcodes for 11_C...
4800/9697 barcodes for 11_C...
5100/9697 barcodes for 11_C...
5400/9697 barcodes for 11_C...
5700/9697 barcodes for 11_C...
6000/9697 barcodes for 11_C...
6300/9697 barcodes for 11_C...
6600/9697 barcodes for 11_C...
6900/9697 barcodes for 11_C...
7200/9697 barcodes for 11_C...
7500/9697 barcode

In [13]:
print(total_time/60)

375.57815090728303


In [14]:
len(results)

88

In [15]:
all_edit_info = pd.concat(results)

In [16]:
all_edit_info.groupby(['ref', 'alt']).count()


barcode    contig  position   read_id    strand  dist_from_end  \
ref alt                                                                    
A   C     8906593   8906593   8906593   8906593   8906593        8906593   
    G     6591721   6591721   6591721   6591721   6591721        6591721   
    N       70768     70768     70768     70768     70768          70768   
    T     3057539   3057539   3057539   3057539   3057539        3057539   
C   A    14629203  14629203  14629203  14629203  14629203       14629203   
    G     2541749   2541749   2541749   2541749   2541749        2541749   
    N       58671     58671     58671     58671     58671          58671   
    T    11856316  11856316  11856316  11856316  11856316       11856316   
G   A     5301682   5301682   5301682   5301682   5301682        5301682   
    C     3358582   3358582   3358582   3358582   3358582        3358582   
    N       61116     61116     61116     61116     61116          61116   
    T     9141855   9141855   9141855   9141855   9141855        9141855   
N   A           2         2         2         2         2              2   
    C           3         3         3         3         3              3   
    G          52        52        52        52        52             52   
    T           5         5         5         5         5              5   
T   A    15033265  15033265  15033265  15033265  15033265       15033265   
    C     4864684   4864684   4864684   4864684   4864684        4864684   
    G     9116317   9116317   9116317   9116317   9116317        9116317   
    N       72004     72004     72004     72004     72004          72004   

         base_quality  mapping_quality  barcode_position  coverage  
ref alt                                                             
A   C         8906593          8906593           8906593   8906593  
    G         6591721          6591721           6591721   6591721  
    N           70768            70768             70768     70768  
    T         3057539          3057539           3057539   3057539  
C   A        14629203         14629203          14629203  14629203  
    G         2541749          2541749           2541749   2541749  
    N           58671            58671             58671     58671  
    T        11856316         11856316          11856316  11856316  
G   A         5301682          5301682           5301682   5301682  
    C         3358582          3358582           3358582   3358582  
    N           61116            61116             61116     61116  
    T         9141855          9141855           9141855   9141855  
N   A               2                2                 2         2  
    C               3                3                 3         3  
    G              52               52                52        52  
    T               5                5                 5         5  
T   A        15033265         15033265          15033265  15033265  
    C         4864684          4864684           4864684   4864684  
    G         9116317          9116317           9116317   9116317  
    N           72004            72004             72004     72004

In [17]:
total_time/60

375.57815090728303

In [18]:
all_edit_info.to_csv('{}/all_edit_info.tsv'.format(output_folder), sep='\t')

# Group by site to get final total edit and coverage counts at each site

In [ ]:
all_edit_info.head()


# Verify C>T ratios

In [ ]:
all_edit_info.groupby(['ref', 'alt']).count()


In [ ]:
all_edit_info.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("All edits")

base_quality_thresh = 15
all_edit_info[all_edit_info.base_quality > base_quality_thresh].groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("Edits with base quality > {}".format(base_quality_thresh))

all_edit_info_filtered = all_edit_info[all_edit_info.base_quality > base_quality_thresh]


In [ ]:
example_new_ct =  all_edit_info_filtered[(all_edit_info_filtered.ref == 'C') & (all_edit_info_filtered.alt == 'T')].sort_values('position')

In [ ]:
len(example_new_ct)

In [ ]:
example_new_ct

# Cells that do have STAMP expressed versus don't...?

In [ ]:
stamp_expression_path = \
'/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_combined/\
4.1_cells_with_middling_stamp/stamp_expression_for_all_used_cells.tsv'

In [ ]:
stamp_expression_df = pd.read_csv(stamp_expression_path, sep='\t', index_col=0)

In [ ]:
stamp_expression_df.Stamp.hist(bins=50)

In [ ]:
all_edit_info_filtered['edit'] = all_edit_info_filtered['ref'] + '>' + all_edit_info_filtered['alt']

In [ ]:
fractions_ct = []
threshs = [0, 0.5, 1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp > thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression above {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct.append(fraction_ct)
    
    

In [ ]:
plt.plot(threshs, fractions_ct)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression minimum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")

In [ ]:
fractions_ct_low = []
threshs = [1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp < thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression below {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct_low.append(fraction_ct)

In [ ]:
plt.plot(threshs, fractions_ct_low)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression maximum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")